In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
session = requests.Session()

params = {'username': "", 'password': "", '_eventId': 'submit', 'submit':'LOGIN', 'lt': 'e1s1'}

In [9]:
def find_total_page_count(bsObj):
    """Returns the total number of postings"""
    posting_count = bsObj.find("div", {"id": "postingsTablePlaceholder"}).find("span", {"class": "badge badge-info"}).get_text().strip()
    return (int(posting_count) - 1) // 100 + 1 #because math module isn't found so math.ceil is unavailable

def find_action_value(bsObj):
    script = bsObj.find('div',{'class': 'tab-content'}).find('script')
    script = str(script)
    function_start_index = script.find('function loadPostingTable')
    first_apostrophe_index = script.find("'", function_start_index)
    second_apostrophe_index = script.find("'", first_apostrophe_index + 1)
    return script[first_apostrophe_index + 1:second_apostrophe_index]

In [10]:
with requests.Session() as s:

    #connection = create_database()
    s.get('https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm')
    p = s.post('https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm', data=params)
    print('==================')

    #go to all postings

    postings_home_page = s.get('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm')

    #get the value for 'Content-Disposition: form-data; name="action"'
    bsObj = BeautifulSoup(postings_home_page._content, "html.parser")
    a = bsObj.find("form", {"id": "widgetSearch"})
    #print(a)
    action_value = bsObj.find("form", {"id": "widgetSearch"}).find("input", {"name": "action"})['value']
    initial_search_action_value = bsObj.find("form", {"id": "widgetSearch"}).find("input", {"name": "initialSearchAction"})['value']
    
    #search data is gotten via the html info from Inspect elements
    search_data = {
    	'action': action_value,
        'initialSearchAction' : initial_search_action_value,
    	'filter': 'all',
    	'widget1Keyword': '',
    	'widget2Keyword': ''
    }

    files = {'': ('', '')}


    search_page = s.post('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm', data=search_data, files=files)

    #####look for the action value in order to go to next page. It's inside
    '''
    function
    loadPostingTable(orderBy, access, customItem, callback, page)
    {
        var
    request = {
        action: '_-_-DtDt7fikcywBvXJ5_GeUVj5FoLjMzUnTyXWgiil8BBQ8Byq4neMkVyIpv2ETW2g6BMkCoBSAjurw07r5KrzL8hKieUeLQOeVo6IIf31ccKMHDBISGKNPo3--CDlArYy0RyzZlG_lKqUNlJAVNNiiWA6pqu1mFPyKBMbqXXSBP_st',
        orderBy: orderBy,
        postings: access,
        page: page,
        initialSearchAction: '',
        searchType: '',
        npfGroup: $('[name=npfGroup]').val(),
                   rand: Math.floor(Math.random() * 100000)
    };
    if (customItem)
    '''
    bsObj = BeautifulSoup(search_page._content, "html.parser")

    total_page_count = find_total_page_count(bsObj)
    action_value_100_postings = find_action_value(bsObj)
    
    rows = []
    for page_count in range(1, total_page_count + 1): #stop at total_page_count + 1 because range is not inclusive
        print ('========================= page_count: ' + str(page_count) + '==========================================')
        data_100_postings = {
            'action': action_value_100_postings,
            'initialSearchAction' : initial_search_action_value,
            'sortDirection': 'Forward',
            'postings': 'infoForPostings',
            'page': str(page_count)
        }

        page = s.post('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm', data=data_100_postings)
        bsObj = BeautifulSoup(page._content, "html.parser")
        tbody = ""
        try:
            tbody = bsObj.select("table#postingsTable > tbody")[0]
        except:
            print('=============== OH NO!! =================')
            print(page.text)

        postings = tbody.find_all('tr')
        #### Look at all jobs on page
        for i in range(len(postings)):

            tds = postings[i].find_all('td')
            #print("Row data: ", tds)
            id = tds[2].get_text() # used to be tds[1], now it's tds[2]. There are changes every once in a while.
            print(id)
            deadline = tds[8].get_text()

            jsFunction = tds[3].find('a')['onclick'] # used to be tds[2], now tds[3]
            #print(jsFunction)

            # This is hard-coded and brittle but I want a quick fix
            # orbisApp.buildForm({'action':'THISISIT!',....})

            startIndex = 30
            endIndex = jsFunction.find("'", startIndex + 1)

            action_value = jsFunction[startIndex : endIndex]
            #print(action_value)

            '''
            #get initialSearchAction
            startIndex = endIndex + 25
            endIndex = jsFunction.find("'", startIndex+1)
            initial_search_action = jsFunction[startIndex : endIndex]
            print(initial_search_action)
            '''


            '''
            path = tds[2].find('a')['href']
            print(path)
            intermediate_page = s.get('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm' + path) #to get action to go to real page
            print (intermediate_page)
            bsObj = BeautifulSoup(intermediate_page._content, "html.parser")
            print(bsObj)
            action_value = bsObj.find("form", {"id": "rform"}).find("input", {"name": "action"})['value']
            '''

            data = {
                'action': action_value,
                'initialSearchAction': initial_search_action_value,
                'searchType':'',
                'accessToPostings':'infoForPostings',
                'postingId': id,
                'npfGroup':'',
                'sortDirection': 'Reverse',
            }


            files = {'': ('', '')}
            posting_page = s.post('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm', data=data, files=files)
            bsObj = BeautifulSoup(posting_page._content, "html.parser")

            #finding company info, the top <h5> for some reason isn't being found. Maybe because of the ::before
            organization_name = ""
            division_name = ""

            try:
                org_table = bsObj.find_all('table')[-1]
                organization_name = (org_table.find_all('tr')[0]).find_all('td')[1].get_text().strip()
                division_name = (org_table.find_all('tr')[1]).find_all('td')[1].get_text().strip()
                #print(organization_name)
                
                app_delivery_table = bsObj.find_all('table')[-2]
                
                trs = bsObj.find('div', {'id': 'postingDiv'}).find('table').find('tbody').find_all('tr', recursive=False)
                app_table_trs = app_delivery_table.find('tbody').find_all('tr', recursive=False)

#                 info = {
# 	                'Organization:': organization_name,
# 	                'Division:': division_name,
# 	                'Work Term:' : '',
# 	                'Job Type:' : '',
# 	                'Job Title:' : '',
# 	                'Number of Job Openings:':'',
# 	                'Job Category (NOC):':'',
# 	                'Level:':'',
# 	                'Region:':'',
# 	                'Job - Address Line One:':'',
# 	                'Job - Address Line Two:':'',
# 	                'Job - City:':'',
# 	                'Job - Province / State:': '',
# 	                'Job - Postal Code / Zip Code (X#X #X#):': '',
# 	                'Job - Country:': '',
# 	                'Work Term Duration:': '',
# 	                'Job Summary:': '',
# 	                'Job Responsibilities:': '',
# 	                'Required Skills:': '',
# 	                'Targeted Degrees and Disciplines:':'',
# 	                'Transportation and Housing:': '',
# 	                'Compensation and Benefits Information:':'',
# 	                'Additional Information:': '',
# 	                'Additional Job Identifiers:':'',
# 	                'Special Job Requirements:':'',
# 	                'Other':''
# 	            }
                
                #FUll time job structure
                info = {
                    'Id': id,
                    'Application Deadline': deadline,
	                'Organization:': organization_name,
	                'Division:': division_name,
	                'Term Posted:' : '',
	                'Position Type:': '',
	                'Job Title:' : '',
	                'Job Openings:':'',
	                'Job Category (NOC):':'',
	                'Region:':'',
	                'Start Date:': '',
	                'Job Summary:': '',
	                'Job Responsibilities:': '',
	                'Required Skills:': '',
	                'Targeted Degrees and Disciplines:': '',
                    'Transportation and Housing:': '',
	                'Compensation and Benefits Information:':'',
	                'Additional Information:': '',
	                'Additional Job Identifiers:':'',
	                'Special Job Requirements:':'',
                    'Application Deadline:': '',
                    'Application Delivery:': '',
                    'If by eMail, send to:': '',
                    'Application Documents Required:':'',
	                'Other':''
	            }


                for tr in trs:
                    tds = tr.find_all('td')
                    type = tds[0].get_text().strip()
                    value = tds[1].get_text().strip()
                    if type in info:
                        info[type] = value
                    else:
                        info['Other'] = value
                        
                for tr in app_table_trs:
                    tds = tr.find_all('td')
                    type = tds[0].get_text().strip()
                    value = tds[1].get_text().strip()
                    if type in info:
                        info[type] = value
                    else:
                        info['Other'] = value                   

                #print("COLLECTED INFO")
                #print(info)
                
                #append to rows array
                rows.append(info)
                
                
            except:
                print("Something bad happened and this job was not stored")


========================= page_count: 1==========================================
127955
126673
128101
128091
128512
128506
128507
127841
127715
127134
128325
128326
127621
127785
128133
128658
128443
128095
125358
127723
126613
126616
129055
127142
127144
127299
127740
124916
126926
126932
126933
126934
126935
126936
126937
126939
126940
128473
129374
127804
126627
124971
124974
128239
127254
129114
128304
128313
128564
128565
128830
128842
128841
128843
128835
127603
128539
128623
128314
129229
128072
128200
128435
128437
128358
127922
127930
127937
127817
129006
129014
129000
129018
129016
129022
129023
129027
129010
129025
127666
127711
127713
127716
127658
127659
128981
129362
128215
128516
128518
128321
128322
128323
129194
129209
127697
127698
127710
128857
128875
========================= page_count: 2==========================================
128858
128874
128872
128873
128859
128871
128868
128867
128860
128865
128866
127868
127878
127983
127989
127990
127620
127633
127634
129

In [11]:
#save to pandas dataframe
df = pd.DataFrame.from_dict(rows, orient="columns")

In [12]:
len(rows)

333

In [13]:
pd.set_option('display.max_columns', None)  
df.head()

,Additional Information:,Additional Job Identifiers:,Application Deadline,Application Deadline:,Application Delivery:,Application Documents Required:,Compensation and Benefits Information:,Division:,Id,"If by eMail, send to:",Job Category (NOC):,Job Openings:,Job Responsibilities:,Job Summary:,Job Title:,Organization:,Other,Position Type:,Region:,Required Skills:,Special Job Requirements:,Start Date:,Targeted Degrees and Disciplines:,Term Posted:,Transportation and Housing:
0,,,"\r\n\t\t\t\t\t\t\t\tSep 10, 2019\r\n\t\t\t\t\t...","Sep 10, 2019 11:59:00 PM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,127955,mike.walker@67.solutions,2174 Computer Programmers and Interactive Medi...,1,Work closely with functional consultants for t...,67 Solutions is a consulting company that spec...,Technical Consultant,67 Solutions Inc,Bachelors\n\n\nMasters,Full-time,ON - Toronto,"SuiteScript, RESTlets and JSON\nExperience wit...",,09/02/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
1,,,"\r\n\t\t\t\t\t\t\t\tJul 17, 2020\r\n\t\t\t\t\t...","Jul 17, 2020 09:00:00 AM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,126673,careers@7dsurgical.com,2173 Software Engineers and Designers,1,"Responsibilities\n\nDesign, develop, and unit ...",At 7D Surgical you will find a diverse and vib...,Software Developer (Junior),7D Surgical,What Makes Us Attractive?\n\nOur mighty team o...,Graduating,ON - Toronto,Qualifications\n\nBachelor's or graduate degre...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
2,,,"\r\n\t\t\t\t\t\t\t\tSep 3, 2019\r\n\t\t\t\t\t\...","Sep 03, 2019 11:59:00 PM",Apply direct to company website,Résumé,,Divisional Office,128101,,1122 Professional Occupations in Business Serv...,1,Highlights of this opportunity:\r\nAre you rea...,A little bit about us:\r\nLaunched in 1999 as ...,Communications Coordinator,Aboriginal Peoples Television Network (APTN),https://workforcenow.adp.com/mascsr/default/md...,Full-time,Western Canada,Your accomplishments and experience include: \...,,05/01/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
3,,,"\r\n\t\t\t\t\t\t\t\tSep 4, 2019\r\n\t\t\t\t\t\...","Sep 04, 2019 11:59:00 PM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,128091,careers@adastragrp.com,2173 Software Engineers and Designers,3,Work with a team of Adastra consultants to com...,Adastra's Nova Consultants are new and recent ...,Nova Consultant - Data Warehousing & Business ...,Adastra Corporation,Please apply by submitting your resume with a ...,Full-time,ON - Toronto,"Computer Science/Computer Engineering degree, ...",,09/03/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
4,,,"\r\n\t\t\t\t\t\t\t\tSep 9, 2019\r\n\t\t\t\t\t\...","Sep 09, 2019 09:00:00 AM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,128512,careers@adastragrp.com,2173 Software Engineers and Designers,2,Work with a team of Adastra consultants in the...,Adastra's Nova Consultants are new and recent ...,Nova Consultant - Data Management,Adastra Corporation,Please apply by submitting your resume with a ...,Full-time,ON - Toronto,"Computer Science, Computer Engineering, or sim...",,09/16/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,


In [14]:
print(df.columns)

Index(['Additional Information:', 'Additional Job Identifiers:',
       'Application Deadline', 'Application Deadline:',
       'Application Delivery:', 'Application Documents Required:',
       'Compensation and Benefits Information:', 'Division:', 'Id',
       'If by eMail, send to:', 'Job Category (NOC):', 'Job Openings:',
       'Job Responsibilities:', 'Job Summary:', 'Job Title:', 'Organization:',
       'Other', 'Position Type:', 'Region:', 'Required Skills:',
       'Special Job Requirements:', 'Start Date:',
       'Targeted Degrees and Disciplines:', 'Term Posted:',
       'Transportation and Housing:'],
      dtype='object')


In [20]:
import fastparquet

In [21]:
#df.to_hdf('data.h5', key="s")
df.to_parquet('Summer2019-Fulltime.parquet', compression="gzip", engine='fastparquet')

In [22]:
test = pd.read_parquet("Summer2019-Fulltime.parquet", engine="fastparquet")

In [23]:
len(test)

333

In [24]:
test.head()

,Additional Information:,Additional Job Identifiers:,Application Deadline,Application Deadline:,Application Delivery:,Application Documents Required:,Compensation and Benefits Information:,Division:,Id,"If by eMail, send to:",Job Category (NOC):,Job Openings:,Job Responsibilities:,Job Summary:,Job Title:,Organization:,Other,Position Type:,Region:,Required Skills:,Special Job Requirements:,Start Date:,Targeted Degrees and Disciplines:,Term Posted:,Transportation and Housing:
0,,,"\r\n\t\t\t\t\t\t\t\tSep 10, 2019\r\n\t\t\t\t\t...","Sep 10, 2019 11:59:00 PM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,127955,mike.walker@67.solutions,2174 Computer Programmers and Interactive Medi...,1,Work closely with functional consultants for t...,67 Solutions is a consulting company that spec...,Technical Consultant,67 Solutions Inc,Bachelors\n\n\nMasters,Full-time,ON - Toronto,"SuiteScript, RESTlets and JSON\nExperience wit...",,09/02/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
1,,,"\r\n\t\t\t\t\t\t\t\tJul 17, 2020\r\n\t\t\t\t\t...","Jul 17, 2020 09:00:00 AM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,126673,careers@7dsurgical.com,2173 Software Engineers and Designers,1,"Responsibilities\n\nDesign, develop, and unit ...",At 7D Surgical you will find a diverse and vib...,Software Developer (Junior),7D Surgical,What Makes Us Attractive?\n\nOur mighty team o...,Graduating,ON - Toronto,Qualifications\n\nBachelor's or graduate degre...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
2,,,"\r\n\t\t\t\t\t\t\t\tSep 3, 2019\r\n\t\t\t\t\t\...","Sep 03, 2019 11:59:00 PM",Apply direct to company website,Résumé,,Divisional Office,128101,,1122 Professional Occupations in Business Serv...,1,Highlights of this opportunity:\r\nAre you rea...,A little bit about us:\r\nLaunched in 1999 as ...,Communications Coordinator,Aboriginal Peoples Television Network (APTN),https://workforcenow.adp.com/mascsr/default/md...,Full-time,Western Canada,Your accomplishments and experience include: \...,,05/01/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
3,,,"\r\n\t\t\t\t\t\t\t\tSep 4, 2019\r\n\t\t\t\t\t\...","Sep 04, 2019 11:59:00 PM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,128091,careers@adastragrp.com,2173 Software Engineers and Designers,3,Work with a team of Adastra consultants to com...,Adastra's Nova Consultants are new and recent ...,Nova Consultant - Data Warehousing & Business ...,Adastra Corporation,Please apply by submitting your resume with a ...,Full-time,ON - Toronto,"Computer Science/Computer Engineering degree, ...",,09/03/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
4,,,"\r\n\t\t\t\t\t\t\t\tSep 9, 2019\r\n\t\t\t\t\t\...","Sep 09, 2019 09:00:00 AM",Apply direct via email,"Cover Letter,Résumé",,Divisional Office,128512,careers@adastragrp.com,2173 Software Engineers and Designers,2,Work with a team of Adastra consultants in the...,Adastra's Nova Consultants are new and recent ...,Nova Consultant - Data Management,Adastra Corporation,Please apply by submitting your resume with a ...,Full-time,ON - Toronto,"Computer Science, Computer Engineering, or sim...",,09/16/2019,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,2019 July to 2020 June,
